# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 12:19PM,249707,12,HH8256.1,Hush Hush,Released
1,Oct 31 2022 12:18PM,249706,19,ADV80007202,"AdvaGen Pharma, Ltd",Released
2,Oct 31 2022 12:18PM,249705,12,U1095,"Uniderm USA, Inc",Released
3,Oct 31 2022 12:17PM,249704,19,ADV80007201,"AdvaGen Pharma, Ltd",Released
4,Oct 31 2022 12:11PM,249702,10,MSP215362,"Methapharm, Inc.",Released
5,Oct 31 2022 12:11PM,249702,10,MSP215363,"Methapharm, Inc.",Released
6,Oct 31 2022 12:11PM,249702,10,MSP215364,"Methapharm, Inc.",Released
7,Oct 31 2022 12:11PM,249702,10,MSP215367,"Methapharm, Inc.",Released
8,Oct 31 2022 12:11PM,249702,10,MSP215368,"Methapharm, Inc.",Released
9,Oct 31 2022 12:11PM,249702,10,MSP215369,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,249702,Released,8
12,249704,Released,1
13,249705,Released,1
14,249706,Released,1
15,249707,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249702,NaN,8.0
249704,NaN,1.0
249705,NaN,1.0
249706,NaN,1.0
249707,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249679,30.0,1.0
249680,0.0,1.0
249683,23.0,0.0
249686,0.0,1.0
249692,0.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249679,30,1
249680,0,1
249683,23,0
249686,0,1
249692,0,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249679,30,1
1,249680,0,1
2,249683,23,0
3,249686,0,1
4,249692,0,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249679,30,1
1,249680,,1
2,249683,23,
3,249686,,1
4,249692,,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 12:19PM,249707,12,Hush Hush
1,Oct 31 2022 12:18PM,249706,19,"AdvaGen Pharma, Ltd"
2,Oct 31 2022 12:18PM,249705,12,"Uniderm USA, Inc"
3,Oct 31 2022 12:17PM,249704,19,"AdvaGen Pharma, Ltd"
4,Oct 31 2022 12:11PM,249702,10,"Methapharm, Inc."
12,Oct 31 2022 12:09PM,249701,10,"Snap Medical Industries, LLC"
28,Oct 31 2022 12:05PM,249700,10,Methapharm-G
29,Oct 31 2022 12:01PM,249699,10,"Senseonics, Incorporated"
30,Oct 31 2022 11:53AM,249698,10,"Virtue Rx, LLC"
31,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 31 2022 12:19PM,249707,12,Hush Hush,,1
1,Oct 31 2022 12:18PM,249706,19,"AdvaGen Pharma, Ltd",,1
2,Oct 31 2022 12:18PM,249705,12,"Uniderm USA, Inc",,1
3,Oct 31 2022 12:17PM,249704,19,"AdvaGen Pharma, Ltd",,1
4,Oct 31 2022 12:11PM,249702,10,"Methapharm, Inc.",,8
5,Oct 31 2022 12:09PM,249701,10,"Snap Medical Industries, LLC",,16
6,Oct 31 2022 12:05PM,249700,10,Methapharm-G,,1
7,Oct 31 2022 12:01PM,249699,10,"Senseonics, Incorporated",,1
8,Oct 31 2022 11:53AM,249698,10,"Virtue Rx, LLC",,1
9,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 12:19PM,249707,12,Hush Hush,1,
1,Oct 31 2022 12:18PM,249706,19,"AdvaGen Pharma, Ltd",1,
2,Oct 31 2022 12:18PM,249705,12,"Uniderm USA, Inc",1,
3,Oct 31 2022 12:17PM,249704,19,"AdvaGen Pharma, Ltd",1,
4,Oct 31 2022 12:11PM,249702,10,"Methapharm, Inc.",8,
5,Oct 31 2022 12:09PM,249701,10,"Snap Medical Industries, LLC",16,
6,Oct 31 2022 12:05PM,249700,10,Methapharm-G,1,
7,Oct 31 2022 12:01PM,249699,10,"Senseonics, Incorporated",1,
8,Oct 31 2022 11:53AM,249698,10,"Virtue Rx, LLC",1,
9,Oct 31 2022 11:20AM,249693,16,Sartorius Bioprocess Solutions,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 12:19PM,249707,12,Hush Hush,1,
1,Oct 31 2022 12:18PM,249706,19,"AdvaGen Pharma, Ltd",1,
2,Oct 31 2022 12:18PM,249705,12,"Uniderm USA, Inc",1,
3,Oct 31 2022 12:17PM,249704,19,"AdvaGen Pharma, Ltd",1,
4,Oct 31 2022 12:11PM,249702,10,"Methapharm, Inc.",8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 12:19PM,249707,12,Hush Hush,1.0,NaN
1,Oct 31 2022 12:18PM,249706,19,"AdvaGen Pharma, Ltd",1.0,NaN
2,Oct 31 2022 12:18PM,249705,12,"Uniderm USA, Inc",1.0,NaN
3,Oct 31 2022 12:17PM,249704,19,"AdvaGen Pharma, Ltd",1.0,NaN
4,Oct 31 2022 12:11PM,249702,10,"Methapharm, Inc.",8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 12:19PM,249707,12,Hush Hush,1.0,0.0
1,Oct 31 2022 12:18PM,249706,19,"AdvaGen Pharma, Ltd",1.0,0.0
2,Oct 31 2022 12:18PM,249705,12,"Uniderm USA, Inc",1.0,0.0
3,Oct 31 2022 12:17PM,249704,19,"AdvaGen Pharma, Ltd",1.0,0.0
4,Oct 31 2022 12:11PM,249702,10,"Methapharm, Inc.",8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1498179,28.0,30.0
12,499412,2.0,0.0
16,249693,1.0,0.0
19,1248471,22.0,23.0
20,249680,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1498179,28.0,30.0
1,12,499412,2.0,0.0
2,16,249693,1.0,0.0
3,19,1248471,22.0,23.0
4,20,249680,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,28.0,30.0
1,12,2.0,0.0
2,16,1.0,0.0
3,19,22.0,23.0
4,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,28.0
1,12,Released,2.0
2,16,Released,1.0
3,19,Released,22.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,30.0,0.0,0.0,23.0,0.0
Released,28.0,2.0,1.0,22.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,30.0,0.0,0.0,23.0,0.0
1,Released,28.0,2.0,1.0,22.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,30.0,0.0,0.0,23.0,0.0
1,Released,28.0,2.0,1.0,22.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()